In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import pickle


In [24]:
def load_data_from_pickle(obj_name):
    '''
    load_data_from_pickle function loads and deserializes an object from a pickle file.

    Args:
        obj_name (str): The path to the pickle file containing the serialized object. 

    Returns:
        obj (pd.DataFrame): The deserialized object that was stored in the pickle file. 

    '''
    with open(obj_name, 'rb') as file:
        obj = pickle.load(file)
    return obj

In [25]:
X_train = load_data_from_pickle("/Users/sharanyasenthil/Downloads/data_processed_data_X_train.pkl")
y_train = load_data_from_pickle("/Users/sharanyasenthil/Downloads/data_processed_data_y_train.pkl")

In [26]:
X_test = load_data_from_pickle("/Users/sharanyasenthil/Downloads/data_processed_data_X_test.pkl")
y_test = load_data_from_pickle("/Users/sharanyasenthil/Downloads/data_processed_data_y_test.pkl")

In [27]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(585664, 18) (585664,) (193589, 18) (193589,)


In [28]:
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])
X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)
train_df = pd.concat([X, y], axis=1)

majority_class = train_df[train_df['SepsisLabel'] == 0]
minority_class = train_df[train_df['SepsisLabel'] == 1]

majority_class_subset = majority_class.sample(n=2*len(minority_class))
train_df = pd.concat([majority_class_subset, minority_class])

X_train = train_df.drop('SepsisLabel', axis=1)
y_train = train_df['SepsisLabel']

X = train_df.drop('SepsisLabel', axis=1)
y = train_df['SepsisLabel']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
X_test.shape, y_test.shape, X_train.shape, y_train.shape

((585664, 18), (193589,), (585664, 18), (585664,))

In [52]:
import requests

def send_post_request(url, data, headers=None):
    """
    Send a POST request to the specified URL.

    Args:
        url (str): The URL of the API endpoint.
        data (dict): The data to send in the body of the POST request.
        headers (dict, optional): Additional headers to include in the request.

    Returns:
        requests.Response: The response from the server.
    """
    if headers is None:
        headers = {
            'Content-Type': 'application/json'
        }
    
    response = requests.post(url, json=data, headers=headers)
    
    if response.status_code == 200:
        print("Request was successful")
    else:
        print(f"Request failed with status code {response.status_code}")
    
    return response


url = "http://localhost:8085/predict" 
data = {
"data": X_test.iloc[:10,:].values.tolist(),
}

response = send_post_request(url, data)
print("Response:", response.json())


Request was successful
Response: {'predictions': [1, 1, 1, 0, 0, 0, 0, 1, 0, 0]}
